<a href="https://colab.research.google.com/github/ValentinaEmili/Sign_language/blob/main/ASL_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

code from

https://github.com/AvishakeAdhikary/Realtime-Sign-Language-Detection-Using-LSTM-Model/blob/main/RealTimeSignLanguageDetection.ipynb

In [ ]:
# mount google drive on colab
from google.colab import drive
drive.mount('/content/drive')

In [2]:
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import numpy as np
import os
#from tensorflow.keras.utils import to_categorical
import torch
import torch.nn.functional as F
from torch.nn import LSTM
from torch.utils.tensorboard import SummaryWriter

In [4]:
# load the JSON data
js_file = pd.read_json("/content/drive/MyDrive/NLP/WLASL_v0.3.json")
folder = "/content/drive/MyDrive/NLP/dataset/"

training_folder = folder + "train/"
test_folder = folder + "test/"
training_video = training_folder + "video/"
test_video = test_folder + "video/"
training_images = training_folder + "images/"
test_images = test_folder + "images/"

#youtube_videos = ['asl5200', 'asllex', 'aslu', 'lillybauer', 'nabboud', 'northtexas', 'scott', 'valencia-asl']

Preprocess the data

In [ ]:
gloss = set()
for image in os.listdir(training_images):
  word, _ = image.split("_")
  gloss.add(word)
for image in os.listdir(test_images):
  word, _ = image.split("_")
  gloss.add(word)
gloss = list(gloss)
label_map = {label: num for num, label in enumerate(gloss)}
X_train, y_train, X_test, y_test = [], [], [], []

for image in os.listdir(training_images):
  np_array = np.load(os.path.join(training_images, image))
  label, _ = image.split("_")
  X_train.append(np_array)
  y_train.append(label_map[label])

for image in os.listdir(test_images):
  np_array = np.load(os.path.join(test_images, image))
  label, _ = image.split("_")
  X_test.append(np_array)
  y_test.append(label_map[label])

num_classes = len(gloss)

X_train = torch.tensor(np.array(X_train), dtype=torch.float32)
X_test = torch.tensor(np.array(X_test), dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)
y_train = F.one_hot(y_train, num_classes=num_classes).int()
y_test = F.one_hot(y_test, num_classes=num_classes).int()

Build and train LSTM Neural Network

In [ ]:
log_dir = "/content/drive/MyDrive/NLP/Logs"
os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

In [ ]:
res = model.predict(X_test)
print("predicted", actions[np.argmax(res[4])])
print("ground truth", actions[np.argmax(y_test[4])])

Save weights

In [ ]:
model.save('./model.h5')
model.save_weights('./model_weights.h5');

In [ ]:
#del model